## First thesis Experiments
In this notbook, I seek to
- explore the data set I've got
- send some openAI request
- run a small sample to get a performance estimate
- vary parameters, models
- run a large sample, potentially from cluster

In [ ]:
!pip install openai
!pip install pandas
!pip install sklearn

In [1]:
import pandas as pd
import pickle as pkl
import openai
from sklearn.metrics import confusion_matrix
import time

# import large file
with open('../ct_df_posts_single_preds.pkl', 'rb') as f:
    pre_df = pkl.load(f)

# shorten to not murder my computer
# df = pre_df.iloc[:50]
#del pre_df

# retrieve API key
import os
api_key = os.environ.get('OPENAI_API_KEY')
# print(api_key)

In [2]:
# sample 50 ads, 50 non-ads
df = pd.concat([pre_df[pre_df["is_ad"]==True].sample(50),
                pre_df[pre_df["is_ad"]==False].sample(50)]).copy()
df.reset_index(drop=True,inplace=True)
del pre_df

In [22]:
# Generate classifications for all samples

completions = []
i=0

for txt in df.loc[:,"caption"]:
    i+=1
    if i%10==0: print(f"Counter at {i}")
    
    # timer to stay within my quota
    if i==34: time.sleep(65)
    if i==68: time.sleep(65)
    
    prompt = f"Judge whether it is likely that the following caption comes from a post that has been sponsored. Try to aim for a 50:50 distribution between True and False.\n\n Post: 'aaaa meu look de hoje é da @cea_brasil ❤️ patrocinadora e dona do look oficial do Rock in Rio! Aproveitem pra acompanhar o perfil da C&A pra ver todos os lookinhos maravilhosos! Amo muito! ❤️'\n Sponsored (True/False): True\n\n Post: 'side by side ~ sisters ❤️'\n Sponsored (True/False): True\n\n Post:{txt}\n Sponsored (True/False):"

    # Generate a response from openai.
    # Flexibly choose ada/babbage/curie/davinci as engine. For davinci, use text-davinci-002.
    response = openai.Completion.create(
        engine="text-davinci-002",
        prompt=prompt,
        max_tokens=5,
        n=1,
        stop=None,
        temperature=0,
    )

    # Save the response text
    completions.append(response["choices"][0]["text"])

Counter at 10
Counter at 20
Counter at 30
Counter at 40
Counter at 50
Counter at 60
Counter at 70
Counter at 80
Counter at 90
Counter at 100


In [23]:
# Make predictions to booleans
completions_as_boolean = [True if response.__contains__("rue") 
                          else False if response.__contains__("als")
                          else "warning" for response in completions]

# check for bad completions
if "warning" in vanilla_TF: print("Warning")

# obtain confusion matrix
matrix = confusion_matrix(df["is_ad"],completions_as_boolean)
matrix

array([[48,  2],
       [19, 31]])

In [6]:
# Example Request OpenAI API
response = openai.Completion.create(
    engine="text-davinci-002",
    prompt="Write a longer essay about what PBL is, how it is applied in different instiutions and discuss advantages and disadvanatages in detail",
    max_tokens=1050,
    n=1,
    stop=None,
    temperature=1,
)